In [6]:
import numpy as np
import os
import glob
import pickle
import music21 as m21
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, stream, note, chord


# First Create The Note Object From Midi Files

In [21]:
#This function transposes all the songs to either cmaj or amin, so that the model does not have to learn every key
def transpose_song(song):
    
    #get the key for the song
    parts = song.getElementsByClass(m21.stream.Part)
    measure = parts[0].getElementsByClass(m21.stream.Measure)
    #key = measure #### This might be working ... ? Can see the key within the music 21 object
    
    
    #estimate the key using m21
    key = song.analyze("key")
        
    #get the interval for the song to transpose by either major or minor
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))
        
    transposed_song = song.transpose(interval)
    
    return transposed_song 
        
    

In [34]:
#notes_from_midi()

In [7]:
def notes_from_midi():
    #create a note objects off all notes, chords, and rests from midi files
    notes = []
    
    for file in glob.glob('midi_files/*.mid'):
        midi = converter.parse(file)
        #Transpose the songs before we create the note object in the next function
        #song = transpose_song(midi) Transposing did not work that well
        #print(midi) -> To make sure it is creating a music stream object
        
        #notes to parse
        notes_to_parse = None
        
        try: #songs have multiple paino parts
            p_parts = instrument.partitionByInstrument(midi)
#             print(len(p_parts)) # just checking to see if the output is what I want
#             print(p_parts[1])
            notes_to_parse = p_parts[0].recurse()
        except: #when note is not a chord
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " + str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name) + " " + str(element.quarterLength))
                
#run this when you are ready to save the notes make sure they are correct first.
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes   

In [1]:
def prepare_notes():
    '''
    prepare the notes to be the input and output used by the network
    
    notes = note object created after parsing the midi files using M21
    
    Output: The input and output sequences to the LSTM network
    
    '''
    
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 100 
    
    #creating all the unique notes for create the dictionary from
    pitches = sorted(set(note for note in notes))
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_input = lstm_input / float(len(pitches))
    #one_hot_encoding lstm_ input
    lstm_output = to_categorical(lstm_output)
    
    return (lstm_input, lstm_output)

In [8]:
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/notes'

In [55]:
len(set(notes))

2424

In [16]:
note_dict = dict((note, number) for number, note in enumerate(pitches))

In [31]:
# pitches = sorted(set(note for note in notes))
# len(pitches)

In [42]:
# pitch = sorted(set(note for note in notes))
# pitch
w,t = prepare_notes()

In [45]:
len(t)

68473

In [48]:
len(set(notes))

2368

In [49]:
t.shape

(68473, 2368)

In [4]:
prepare_notes()

FileNotFoundError: [Errno 2] No such file or directory: 'data/notes'

In [9]:
notes_from_midi()

['rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.9 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.5 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.0 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.9 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.5 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.0 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.9 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.5 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.0 0.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '7.11 0.5',
 'F3 0.5',
 'C5 0.5',
 'C4 0.5',
 'D5 0.5',
 'rest 0.5',
 '5.9.0 2.25',
 'C4 0.5',
 'rest 0.5',
 'C4 0.5',
 'rest 0.5',
 'C4 0.5',
 'rest 1.25',
 'F3 0.5',
 'C4 0.5',
 '4.9 0.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'E5 0.25',
 '2.5 0.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'D5 0.25',
 '9.0 0.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'C5 0.25',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.9 0.25',
 'rest 0.5',
 'F4 0.25',


In [ ]:
model.add(MultiHead(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True))num_heads=2, key_dim=2))